In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from scipy.stats import linregress, t as t_dist
import pandas as pd
import os
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
dtype=torch.float32
glist = [1.1, 1.3, 1.5]
delta_t = 0.1 ; step_t = 2000
N = 6000
tail_range =201 ; num_in = 20 ; num_out = 100
Ace_RNN_Mean, Ace_RNN_Std = [], []
Slope_List_RNN, Intercept_List_RNN, Stderr_List_RNN = [], [], []
master_seed = 2025
torch.manual_seed(master_seed)
np.random.seed(master_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(master_seed)
seed_tabel = np.random.randint(0, 2**31-1, size=(num_out, num_in))
phi = torch.tanh
T = step_t * delta_t

"""------------------------ RNN Simulation ---------------------------"""
for g in glist:
    print(f"\n=== Simulating g = {g:.2f} ===")
    slopes,intercepts,smeans=[],[],[]
    for i in tqdm(range(num_out), desc=f"Simulating g={g:.2f}"):
        seed_J = int(seed_tabel[i,0])
        torch.manual_seed(seed_J)
        np.random.seed(seed_J)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed_J)

        J = torch.randn((N, N),dtype=dtype, device=device) * (g / np.sqrt(N))
        speed_s = torch.zeros((num_in, step_t), dtype=dtype,device=device)
        s_rnn = torch.zeros((num_in, step_t + 1), dtype=dtype,device=device)
    
        for j in range(num_in):
            seed_X = int(seed_tabel[i,j])
            torch.manual_seed(seed_X)
            np.random.seed(seed_X)
            X =  torch.randn(N, dtype=dtype, device=device)
            for t in range(step_t):
                dx_dt = -X + J @ phi(X)
                X +=  delta_t * dx_dt
                speed_s[j,t] = torch.sqrt(torch.sum(dx_dt ** 2) / N).cpu()
                
        s_rnn[:, 1:] = torch.cumsum(speed_s * delta_t, dim=1)
        s_mean = torch.mean(s_rnn, dim=0).cpu().numpy()
        s_std = torch.std(s_rnn, dim=0).cpu().numpy()
       
        x_fit = np.arange(step_t - tail_range + 1, step_t + 1) * delta_t
        y_fit = s_mean[-tail_range:]
        result = linregress(x_fit, y_fit)
        slopes.append(result.slope)
        intercepts.append(result.intercept)
        smeans.append(s_mean)

        del X,J
        torch.cuda.empty_cache()
        gc.collect()

    s_means = np.array(smeans)
    slopes = np.array(slopes)
    intercepts = np.array(intercepts)
    slope_mean = np.mean(slopes)
    slope_std = np.std(slopes)
    intercept_mean = np.mean(intercepts)

    s_rnn_mean = np.mean(s_means, axis=0)
    s_rnn_std = np.std(s_means, axis=0, ddof=1)
    n_samples = s_means.shape[0]
    s_rnn_err = s_rnn_std / np.sqrt(n_samples)
    Ace_RNN_Mean.append(s_rnn_mean)
    Ace_RNN_Std.append(s_rnn_std)

    alpha = 0.05
    dof = tail_range - 2
    t_crit = t_dist.ppf(1 - alpha / 2, df=dof)
    ci_rnn = t_crit * s_rnn_err
    print(f"RNN slope (last 50 steps): {slope_mean.item():.4f} + -{slope_std.item():.4f}")
 
    Slope_List_RNN.append(slope_mean)
    Intercept_List_RNN.append(intercept_mean)
    Stderr_List_RNN.append(slope_std)
    del slopes,intercepts, smeans 
    torch.cuda.empty_cache()

try:
    folder = os.path.dirname(os.path.abspath(__file__))
except NameError:
    folder = os.getcwd()
filename = f"RNNSIM_N{N}.csv"
RNN_path = os.path.join(folder, filename)
RNN_data = {
    'Arc_RNN_Mean': [x.tolist() for x in Ace_RNN_Mean],
    'Arc_RNN_Std':  [x.tolist() for x in Ace_RNN_Std],
    'Slope_List_RNN': Slope_List_RNN,
    'Intercept_List_RNN': Intercept_List_RNN,
    'Stderr_List_RNN': Stderr_List_RNN
}
pd.DataFrame(RNN_data).to_csv(RNN_path, index=False)
print(f"Saved to {RNN_path}")


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import math
from scipy.optimize import root_scalar
import warnings
import os
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
torch.cuda.empty_cache()
MC_SAMPLES = 10_000_000  
MC_BATCH = 200_000      
glist = [1.1, 1.3, 1.5] 
glist = torch.tensor(glist, device=device)

@torch.jit.ignore
def IntPhi_torch(x):
    return torch.log(torch.cosh(x))
@torch.jit.ignore
def phi_torch(x):
    return torch.tanh(x)

def batched_expectations(fn, total_samples, batch_size, device=device, dtype=torch.float32, rng_seed=None):
    if rng_seed is not None:  
        torch.manual_seed(rng_seed)
        if device.type == "cuda":
            torch.cuda.manual_seed_all(rng_seed)
    n_batches = int(math.ceil(total_samples / batch_size))
    acc = 0.0
    acc_count = 0
    for b in range(n_batches):
        current = batch_size if (b < n_batches - 1) else (total_samples - batch_size * (n_batches - 1))    # 生成两个独立正态向量（按需要）
        batch = torch.randn(current, device=device, dtype=dtype)
        batch_res = fn(batch) 
        acc += batch_res.detach().sum().item()
        acc_count += current
    mean = acc / acc_count
    return mean

def Equation_D0_ggt1(x_tensor, g, total_samples=MC_SAMPLES, batch_size=MC_BATCH, rng_seed=None):
    with torch.no_grad():
        x = float(torch.abs(x_tensor[0]).item())
        term1 = -0.5 * x ** 2      
        sqrt_x = math.sqrt(x) if x > 0 else 0.0
        def fn_IntPhi2(z_batch):
            vals = IntPhi_torch(sqrt_x * z_batch)  
            return (vals ** 2).sum()
        mean_IntPhi2 = batched_expectations(fn_IntPhi2, total_samples, batch_size, device=device, dtype=torch.float32, rng_seed=rng_seed)
        term2 = (g.item() ** 2) * mean_IntPhi2

        def fn_IntPhi(z_batch):
            vals = IntPhi_torch(sqrt_x * z_batch)
            return vals.sum()
        mean_IntPhi = batched_expectations(fn_IntPhi, total_samples, batch_size, device=device, dtype=torch.float32, rng_seed=(None if rng_seed is None else rng_seed + 1))
        term3 = (g.item() ** 2) * (mean_IntPhi ** 2)
        return float((term1 + term2 - term3))

def Solve_Delta0_ggt1(g, mc_samples=MC_SAMPLES, mc_batch=MC_BATCH):
    def scalar_eqn(delta):
        if delta <= 0:
            return 1e-6
        x_tensor = torch.tensor([delta], device=device)
        return Equation_D0_ggt1(x_tensor, g, total_samples=mc_samples, batch_size=mc_batch)

    sigma = g - 1.0
    approx = max(sigma - 7/6 * sigma**2, 1e-7)
    a = max(1e-8, approx / 10)
    b = approx * 4.0
    if a >= b:
        a = 1e-8
        b = max(1e-3, approx + 0.1)

    result = root_scalar(scalar_eqn, method='brentq', bracket=[a, b], xtol=1e-11)
    if result.converged and abs(scalar_eqn(result.root)) < 1e-4:
        return result
    else:
        raise RuntimeError("Delta0 求解失败或误差过大")

def Calculate_Tau0vsg(num_tries=1000, mc_samples=MC_SAMPLES, mc_batch=MC_BATCH):
    g_arr = glist
    Gamma0_list = []
    Delta0_list = []
    funResult = []
    Delta0_std = []
    Gamma0_std = []
    sqrt_Gamma0_mean = []
    sqrt_Gamma0_std = []

    for g in g_arr:
        Deltas = []
        Gammas = []
        for _ in range(num_tries):
            try:
                r = Solve_Delta0_ggt1(g, mc_samples, mc_batch)
                Delta0 = r.root
                sqrt_D0 = math.sqrt(abs(Delta0))
                def fn_phi2(z_batch):
                    vals = phi_torch(sqrt_D0 * z_batch)
                    return (vals ** 2).sum()
                mean_phi2 = batched_expectations(fn_phi2, mc_samples, mc_batch, device=device, dtype=torch.float32)
                Gamma0 = (g.item() ** 2) * mean_phi2 - Delta0
                if Gamma0 < 0:
                    continue
                Deltas.append(Delta0)
                Gammas.append(Gamma0)
            except Exception:
                continue
            finally:
                torch.cuda.empty_cache()
                gc.collect()

        if len(Deltas) == 0:
            continue

        Delta0_arr = np.array(Deltas)
        Gamma0_arr = np.array(Gammas)
        Delta0_list.append(Delta0_arr.mean())
        Gamma0_list.append(Gamma0_arr.mean())
        Delta0_std.append(Delta0_arr.std())
        Gamma0_std.append(Gamma0_arr.std())
        sqrt_Gamma0_mean.append(np.sqrt(Gamma0_arr).mean())
        sqrt_Gamma0_std.append(np.sqrt(Gamma0_arr).std())

        fun_val = Equation_D0_ggt1(torch.tensor([Delta0_arr.mean()], device=device), g, total_samples=mc_batch, batch_size=min(mc_batch, 20000))
        funResult.append(fun_val)

    return Gamma0_list, g_arr.cpu().numpy(), Delta0_list, funResult, Delta0_std, Gamma0_std, sqrt_Gamma0_mean, sqrt_Gamma0_std

Gamma0_history, g_arr, Delta_0_history, funResult, Delta0_std, Gamma0_std, sqrt_Gamma0_mean, sqrt_Gamma0_std = Calculate_Tau0vsg()

try:
    folder = os.path.dirname(os.path.abspath(__file__))
except NameError:
    folder = os.getcwd()

filename = f"DMFT_kinetic_vs_g{g_arr[0]:.2f}to{g_arr[-1]:.2f}.csv"
dmft_path = os.path.join(folder, filename)
dmft_data = {
    'g': g_arr,
    'Gamma0': Gamma0_history,
    'Delta0': Delta_0_history,
    'funResult': funResult,
    'Delta0_std': Delta0_std,
    'Gamma0_std': Gamma0_std,
    'sqrt_Gamma0_mean': sqrt_Gamma0_mean,
    'sqrt_Gamma0_std': sqrt_Gamma0_std
}
df_dmft = pd.DataFrame(dmft_data)
df_dmft.to_csv(dmft_path, index=False)
print(f"Saved {dmft_path}")
